In [1]:
import sys
import re
import pandas as pd
import spacy
import string

from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
separator=';'
com_engine='python'

uri = 'mongodb://localhost:27017/'
database = 'zs_database'
table = 'stories'
filename = 'zs200.csv'

In [4]:
df = pd.read_csv(filename, sep=separator, engine=com_engine)
print(df.shape)
df.head()

(302, 17)


,id,title,abstract,content,plain_text,is_published,is_pre_published,consent_competition_rules,created_at,updated_at,votes_count,user_id,user_created_at,user_age,user_gender,user_city,user_country
0,19,Radio Über-All,Dich quälen Fragen zum Klimawandel? \r\n\r\nSc...,<div><br></div><h1><strong>Radio Über-All send...,Radio Über-All sendet 6 mal Zukunftsmusik! Am ...,True,NaN,True,2020-01-01 20:23:22 +0100,2020-04-03 13:50:55 +0200,0,15,2020-01-01 20:12:55 +0100,48.0,weiblich,Frankfurt am Main,DE
1,21,Unser Dorf pflanzt Sauerstoff,"Mathilda wünscht sich Schnee, um mit ihren Fre...",<div>Unser Dorf pflanzt Sauerstoff<br><br></di...,Unser Dorf pflanzt SauerstoffMathilda schaute ...,True,True,True,2020-01-02 10:09:27 +0100,2020-02-18 10:55:42 +0100,14,27,2020-01-02 10:02:30 +0100,54.0,weiblich,Münster,DE
2,23,Rettung für den Wald und seine Bewohner,Rettung für den Wald und seine Bewohner \r\n \...,<div>Rettung für den Wald und seine Bewohner&n...,Rettung für den Wald und seine Bewohner Lulu ...,False,True,True,2020-01-02 17:41:48 +0100,2020-01-02 18:09:53 +0100,2,16,2020-01-01 20:22:25 +0100,NaN,NaN,NaN,NaN
3,24,Das Märchen vom Klimawandel,"Die Stiefmutter fährt SUV, dem Wolf wird der W...",<div>Es war einmal vor gar nicht so langer Zei...,"Es war einmal vor gar nicht so langer Zeit, da...",True,True,True,2020-01-02 21:25:04 +0100,2020-02-22 16:11:28 +0100,4,14,2020-01-01 20:11:43 +0100,27.0,weiblich,Ravensburg,DE
4,25,Die drei Geister des Klimawandels,"Bernd ist kein schlechter Kerl, aber er lässt ...","<div>Das Beste an Partys ist das Buffet, finde...","Das Beste an Partys ist das Buffet, findet Ber...",True,True,True,2020-01-03 22:05:22 +0100,2020-01-29 20:07:43 +0100,42,13,2020-01-01 20:09:00 +0100,33.0,weiblich,Bremen,DE


In [8]:
df2=df[df['is_published'] == True]

In [13]:
print(len(df2))
df2.head()

200


,id,title,abstract,content,plain_text,is_published,is_pre_published,consent_competition_rules,created_at,updated_at,votes_count,user_id,user_created_at,user_age,user_gender,user_city,user_country
0,19,Radio Über-All,Dich quälen Fragen zum Klimawandel? \r\n\r\nSc...,<div><br></div><h1><strong>Radio Über-All send...,Radio Über-All sendet 6 mal Zukunftsmusik! Am ...,True,NaN,True,2020-01-01 20:23:22 +0100,2020-04-03 13:50:55 +0200,0,15,2020-01-01 20:12:55 +0100,48.0,weiblich,Frankfurt am Main,DE
1,21,Unser Dorf pflanzt Sauerstoff,"Mathilda wünscht sich Schnee, um mit ihren Fre...",<div>Unser Dorf pflanzt Sauerstoff<br><br></di...,Unser Dorf pflanzt SauerstoffMathilda schaute ...,True,True,True,2020-01-02 10:09:27 +0100,2020-02-18 10:55:42 +0100,14,27,2020-01-02 10:02:30 +0100,54.0,weiblich,Münster,DE
3,24,Das Märchen vom Klimawandel,"Die Stiefmutter fährt SUV, dem Wolf wird der W...",<div>Es war einmal vor gar nicht so langer Zei...,"Es war einmal vor gar nicht so langer Zeit, da...",True,True,True,2020-01-02 21:25:04 +0100,2020-02-22 16:11:28 +0100,4,14,2020-01-01 20:11:43 +0100,27.0,weiblich,Ravensburg,DE
4,25,Die drei Geister des Klimawandels,"Bernd ist kein schlechter Kerl, aber er lässt ...","<div>Das Beste an Partys ist das Buffet, finde...","Das Beste an Partys ist das Buffet, findet Ber...",True,True,True,2020-01-03 22:05:22 +0100,2020-01-29 20:07:43 +0100,42,13,2020-01-01 20:09:00 +0100,33.0,weiblich,Bremen,DE
6,27,"Vaiko, der vegetarische Vampir",Im idyllischen Transsilvanien lebt Vaiko mit s...,"<div>Endlose Laubwälder, saftige Weiden, Wildb...","Endlose Laubwälder, saftige Weiden, Wildblumen...",True,True,True,2020-01-05 14:44:21 +0100,2020-03-18 14:25:04 +0100,23,37,2020-01-03 11:01:53 +0100,34.0,weiblich,Ammeldingen bei Neuerburg,DE


In [32]:
from collections import defaultdict

wordCount = defaultdict(int)
punctuation = set(string.punctuation)
data_test=[]
corpus = []

data = list(zip(df2['id'],df2['plain_text']))

for d in data:
    r = ''.join([c for c in d[1].lower() if not c in punctuation])
    words = r.strip().split() # tokenizes
    corpus.append(words)
    
print(len(corpus))
sum=0
for c in corpus:
    sum+=len(c)
print('average length of the story:',(sum/200))

#TODO:Eliminate Germen Stopwords

200
average length of the story: 3938.755
